In [1]:
from __future__ import print_function

In [2]:
import sys
sys.path.insert(0, '../src/')
import generator

import numpy as np
import cplex

# Instance generator for Demand-Response problems

Generates instances of residential Demand Response problems, formulated as Mixed-Integer Linear Programs (MILP).

These models are described in Anjos, Lodi, Tanneau, _A Decentralized Framework for the Optimal Coordination of Distributed Energy Resources_


The considered problems are MILPs of the form:

$$
\begin{array}{rrl}
\displaystyle \min_{y, x} &  q^{T}y + \sum_{r} c_{r}^{T} x_{r}\\
s.t.  &  My + \sum_{r} A_{r} x_{r} &= b\\
& x_{r} &\in \mathcal{X}_{r}
\end{array}
$$
where $x_{r} \in \mathcal{X}_{r}$ encodes linear constraints and integrality requirements on $x_{r}$.

## Hyper-parameters

In [3]:
n_hh = 1024  # number of households

t_beg = 408  # beginning of the time horizon
t_horizon = 24  # number of time-steps

seed = 15984293  # random seed for the example

In [4]:
p_renew = 0.10  # proportion of households that contain PV+EV+Battery

# ownership rates of the considered devices
own_rates = {
    'pv': p_renew,  # rooftop solar PV
    'dishwasher': 0.65,
    'clothes_washer': 0.9,
    'clothes_dryer': 0.75,
    'ev': p_renew,  # electric vehicle
    'heating': 0.6,  # space heating
    'battery': p_renew  # home battery
}

## Generate instance

In [5]:
# generate instance
m = generator.generate_instance(n_hh, t_beg, t_horizon, own_rates, seed=seed)

# m.write('dr_model.mps')  # un-comment this line to export the model in MPS format

In [6]:
# print(stats
print('Variables (all) :', m.variables.get_num())
print('   > Binary     :', m.variables.get_num_binary())
print('   > Continuous :', m.variables.get_num()-m.variables.get_num_binary())
print('Constraints     :', m.linear_constraints.get_num())

Variables (all) : 202023
   > Binary     : 77247
   > Continuous : 124776
Constraints     : 152338


## Solve (using CPLEX)

In [7]:
# Restrict use of resources
m.parameters.timelimit.set(120)
m.parameters.threads.set(1)

# Solve model
m.solve()

CPXPARAM_TimeLimit                               120
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_CallbackReducedLP          0
Tried aggregator 4 times.
MIP Presolve eliminated 42978 rows and 24617 columns.
MIP Presolve modified 211 coefficients.
Aggregator did 68654 substitutions.
Reduced MIP has 40706 rows, 108752 columns, and 291203 nonzeros.
Reduced MIP has 56763 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.57 sec. (656.07 ticks)
Found incumbent of value 1353.342906 after 0.88 sec. (953.77 ticks)
Probing time = 0.02 sec. (16.53 ticks)
Tried aggregator 1 time.
Reduced MIP has 40706 rows, 108752 columns, and 291203 nonzeros.
Reduced MIP has 56763 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.21 sec. (194.74 ticks)
Probing time = 0.02 sec. (16.53 ticks)
Clique table members: 2414.
MIP emphasis: balance optimali